Notebook 2
==============
Data Cleaning and Feature Engineering

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import pyfra
import seaborn as sns

# Data import & pulling

We have imported the data and separated them into four different categories: characteristics,places,users,vehicles.

In [2]:
french_categories = {'characteristics': 'caracteristiques', 'places':'lieux', 'users':'usagers', 'vehicles':'vehicules'}
data_categories = french_categories.keys()
categories_dict = dict(zip(data_categories, [0,0,0,0]))

In [3]:
# Define the function that reads the raw data for the specified time range
def read_csv_of_year(start_year, end_year, separators, name_separator='_'):
    if len (separators)<4:
        separators = [separators]*4
        
    df_dict = {}
    for year in range(start_year,end_year+1):
        this_year_str = str(year)
        # Data Category
        this_df_dict = {}        
        for this_category, this_sep in zip(data_categories, separators):
            # We need the French name of the category for the filename
            this_french_category = french_categories[this_category]
            this_file_path_and_name = '../data/'+this_year_str+'/' + this_french_category+name_separator+this_year_str+'.csv'
            this_df_dict[this_category] = pd.read_csv(this_file_path_and_name, encoding='latin-1', sep=this_sep, low_memory=False)
        df_dict[year] = this_df_dict
    return df_dict

# Import years
df_dict = {}
df_dict.update(read_csv_of_year(2005, 2008, separators=','))
df_dict.update(read_csv_of_year(2009,2009, separators=['\t', ',', ',', ',']))
df_dict.update(read_csv_of_year(2010, 2016, separators=','))
df_dict.update(read_csv_of_year(2017, 2018, separators=',', name_separator='-'))
df_dict.update(read_csv_of_year(2019, 2021, separators=';', name_separator='-'))

In [4]:
dict_of_category_dfs = {}
for this_category in data_categories:
    dict_of_category_dfs[this_category] = pd.concat([df_dict[year][this_category] for year in range(2005,2022)], ignore_index=True)

characteristics = dict_of_category_dfs['characteristics']
places = dict_of_category_dfs['places']
users = dict_of_category_dfs['users']
vehicles = dict_of_category_dfs['vehicles']

# Data Cleaning
We will perform some of the cleaning of the data on the individual datasets. Not all cleaning is possible before merging the datasets, so there will be a second round of cleaning.

## Calculate the percentage of missing values for each dataframe

In [5]:
def na_percentage(df):
  return df.isna().sum() *100 / len(df)

In [6]:
for this_category, df in dict_of_category_dfs.items():
    print(this_category+'\n', na_percentage(df),'\n')

characteristics
 Num_Acc     0.000000
an          0.000000
mois        0.000000
jour        0.000000
hrmn        0.000000
lum         0.000000
agg         0.000000
int         0.000000
atm         0.006509
col         0.001694
com         0.000178
adr        12.774581
gps        57.198251
lat        43.427300
long       43.427656
dep         0.000000
dtype: float64 

places
 Num_Acc     0.000000
catr        0.000089
voie        9.556239
v1         56.690481
v2         95.419907
circ        0.140339
nbv         0.243498
pr         42.394463
pr1        42.556646
vosp        0.246173
prof        0.173774
plan        0.203375
lartpc     19.511203
larrout    10.060977
surf        0.171545
infra       0.482181
situ        0.443128
env1       15.027314
vma        85.457720
dtype: float64 

users
 Num_Acc         0.000000
place           4.906241
catu            0.000000
grav            0.000000
sexe            0.000000
trajet          0.019684
secu           16.893474
locp            2.245798

## Users Dataset

In [7]:
#Grav
users.grav.replace(to_replace=-1,value=1,inplace=True)

In [8]:
#Place

users.place.value_counts()
users.place.fillna(1,inplace=True) #replace is with mode
users.place.replace(to_replace=-1,value=0,inplace=True) #-1 is unassigned , will put 0 unknown #Same result

In [9]:
#Trajet

users.trajet.replace(to_replace=-1,value=0,inplace=True) #-1 is unassigned , will put 0 unknown #Same result
users.trajet.fillna(5,inplace=True) #replace is with mode

In [10]:
#locp

users.locp.replace(to_replace=-1,value=0,inplace=True) #-1 is unassigned , will put 0 unknown #Same result
users.locp.value_counts()
users.locp.fillna(0,inplace=True) #replace is with mode

In [11]:
#actp

users.actp.replace(to_replace=['B'],value=0,inplace=True)#-1,B is unassigned , will put 0 unknown #Same result
users.actp.replace(to_replace=' -1',value=0,inplace=True)
users.actp.replace(to_replace=['A'],value=8,inplace=True) #A is coming in/out of vehicule , will put 8 instead (int)
users.actp.fillna(0,inplace=True) #replace is with mode

users.actp.replace(to_replace='0',value=0,inplace=True)
users.actp.replace(to_replace='1',value=1,inplace=True)
users.actp.replace(to_replace='2',value=2,inplace=True)
users.actp.replace(to_replace='3',value=3,inplace=True)
users.actp.replace(to_replace='4',value=4,inplace=True)
users.actp.replace(to_replace='5',value=5,inplace=True)
users.actp.replace(to_replace='6',value=6,inplace=True)
users.actp.replace(to_replace='7',value=7,inplace=True)
users.actp.replace(to_replace='8',value=8,inplace=True)
users.actp.replace(to_replace='9',value=9,inplace=True)

In [12]:
#etatp

users.etatp.replace(to_replace=-1,value=0,inplace=True) #-1, is unassigned , will put 0 unknown #Same result
users.etatp.isna().sum() #119291
users.etatp.value_counts() 
users.etatp.fillna(0,inplace=True) #replace is with mode

In [13]:
#an_nais

users.an_nais.isna().sum() #10080
users.an_nais.value_counts() 
users.an_nais.fillna(1986.0,inplace=True)
#replace is with mode #the first 5 values of value_counts are close to each other


In [14]:
#Sex 
users.sexe.replace(to_replace=-1,value=1,inplace=True)

### Fixing incoherency of 'secu' Variable
Safety equipment until 2018 was in 2 variables: existence and use.

From 2019, it is the use with up to 3 possible equipments for the same user
(especially for motorcyclists whose helmet and gloves are mandatory).

#secu1
The character information indicates the presence and use of the safety equipment:
-1 - No information
0 - No equipment
1 - Belt
2 - Helmet
3 - Children device
4 - Reflective vest
5 - Airbag (2WD/3WD)
6 - Gloves (2WD/3WD)
7 - Gloves + Airbag (2WD/3WD)
8 - Non-determinable
9 - Other

#secu2
The character information indicates the presence and use of the safety equipment

#secu3
The character information indicates the presence and use of safety equipment

In [15]:
#Security 
users.drop(columns='secu3',inplace=True)

In [16]:
#Searching for the exact index where the year 2019 starts to accuratly study secu factors
X = users.loc[users['Num_Acc'].astype(str).str.startswith("2019")].index[0]

In [17]:
#secu has some missing values for older years , must fill with mode before continuing

users.secu[:X].fillna(value=1,inplace=True)

In [18]:
users['SecuA'] = ((users.secu - users.secu%10)/10) #Type of security Used
users['SecuB']=   users.secu%10  # Was the security used or No

#SecuA is not very important we will focus on secuB if the item was used or not to facilitate study

In [19]:
# 0 is unknown change to others 9 
users.SecuA.replace(to_replace=0,value=9,inplace=True)

In [20]:
# 2-No 3-UnDeterminable 0-Unknown , change all to 0 not used
users.SecuB.replace(to_replace=3,value=0,inplace=True)
users.SecuB.replace(to_replace=2,value=0,inplace=True)

For Secu1-secu2 
We will gather all usage for security variable with 1 value {1} , and if there is no safety we will use {0} , No need to take multiple security parameters (secu2) we will noly take into consideration 1 security variable for comparibility with earlier years.

In [21]:
users.secu1.replace(to_replace=[2,3,4,5,6,7,8,9],value=1,inplace=True)
users.secu1.replace(to_replace=[-1],value=0,inplace=True)

In [22]:
users.secu1.value_counts() 

1.0    335524
0.0     31901
Name: secu1, dtype: int64

In [23]:
users.SecuB.value_counts()

1.0    1672237
0.0     469958
Name: SecuB, dtype: int64

Now both secu1 and secuB are same format we need to merge them into 1 column (for all years)

In [24]:
#must add iloc
users['Security']=0
users['Security'][:X] = users.SecuB[:X]
users['Security'][X+1:] = users.secu1[X+1:]

C:\Users\Louna\AppData\Local\Temp\ipykernel_148\2256942316.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['Security'][:X] = users.SecuB[:X]
C:\Users\Louna\AppData\Local\Temp\ipykernel_148\2256942316.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  users['Security'][X+1:] = users.secu1[X+1:]


In [25]:
#To drop unneeded columns
#copy this to begining of df 3 before modeling for df
#users = users.drop(columns=['secu','secu1','secu2','SecuA','SecuB'])

In [26]:
na_percentage(users)

Num_Acc         0.000000
place           0.000000
catu            0.000000
grav            0.000000
sexe            0.000000
trajet          0.000000
secu           14.640663
locp            0.000000
actp            0.000000
etatp           0.000000
an_nais         0.000000
num_veh         0.000000
id_vehicule    85.359337
secu1          85.359337
secu2          85.359337
SecuA          14.640663
SecuB          14.640663
Security        0.000000
dtype: float64

### Translating the variable names from French to English

In [27]:
users = users.rename(columns = {'catu' : 'User_category',
                                'grav' : 'Severity' , #Severity of accident
                                'sexe' : 'Sex' , #Sex of Driver
                                'trajet' : 'Trajectory' , 
                                'locp' : 'LOCP' , #localisation of pedestrian
                                'actp' : 'ACTP' , #action of pedestrian
                                'etatp' : 'StateP' , #State of pedestrian during accident
                                'an_nais' : 'YoB' , #Year of Birth
                               })
users.columns

#change type to int
users.place = users.place.astype(int)
users.Trajectory = users.Trajectory.astype(int)
users.LOCP = users.LOCP.astype(int)
users.ACTP = users.ACTP.astype(int)
users.StateP = users.StateP.astype(int)
users.YoB = users.YoB.astype(int)

## Places Dataset

### Dropping unwanted columns , which are v1 , v2, vma, voie, env1

In [28]:
# Droped 'Unnamed: 0','v1','v2','vma', because they contained no information.

places = places.drop(['v1','v2','vma','voie','env1'], axis = 1)

### French to English Variables

In [29]:
# Change french names against english names.

places = places.rename(columns = {'catr' : 'Rd_Cat', 'circ' : 'Traf_Direct' , 'nbv' : 'Lanes' ,
                           'pr' : 'Landmark' , 'pr1' : 'Dist_to_Landmark', 'vosp' : 'Add_Lanes', 'prof' : 'Rd_Prof' ,
                          'plan' : 'Rd_Plan' , 'lartpc' : 'Gre_Verge' , 'larrout' : 'Rd_Width', 'surf' : 'Rd_Cond',
                          'infra' : 'Envinmt' , 'situ' : 'Pos_Acc'})
places.head()

,Num_Acc,Rd_Cat,Traf_Direct,Lanes,Landmark,Dist_to_Landmark,Add_Lanes,Rd_Prof,Rd_Plan,Gre_Verge,Rd_Width,Rd_Cond,Envinmt,Pos_Acc
0,200500000001,3.0,2.0,2.0,1.0,430.0,0.0,1.0,1.0,0,63,1.0,0.0,1.0
1,200500000002,2.0,0.0,2.0,0.0,0.0,1.0,1.0,1.0,0,100,1.0,0.0,5.0
2,200500000003,2.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0,0,2.0,0.0,5.0
3,200500000004,3.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,0,0,1.0,0.0,1.0
4,200500000005,3.0,2.0,2.0,24.0,630.0,0.0,1.0,3.0,0,59,2.0,0.0,3.0


### Changing Nans with Zeros

In [30]:
# There is no value = 0 assigned to information in the places data set. 
# Zeros are used in the cleaned data set as a feature to identify original Nans
# and to keep the data set with as much information as possible.

places = places.fillna({'Rd_Cat':0, 'Traf_Direct': 0, 'Lanes':0, 'Add_Lanes':0, 'Rd_Prof':0,'Rd_Plan':0,
                        'Rd_Cond':0, 'Envinmt':0, 'Pos_Acc':0})

### Changing needed "object" Variables to "int" Variables

In [31]:
# Convert 'object' Variables to 'float' Variables

object_list = ['Landmark', 'Dist_to_Landmark', 'Gre_Verge', 'Rd_Width']

places[object_list] = places[object_list].apply(pd.to_numeric, errors='coerce', axis=1)

# Replace empty cells with 'Nans'

places.replace('', np.nan).copy()

# Fill 'Nans' with 0

places = places.fillna({'Landmark':0, 'Dist_to_Landmark': 0, 'Gre_Verge':0, 'Rd_Width':0})

# Convert 'float' Variables to 'int' Variables

float_list = ['Rd_Cat', 'Traf_Direct', 'Lanes', 'Landmark','Dist_to_Landmark', 'Add_Lanes', 'Rd_Prof', 'Rd_Plan',
              'Gre_Verge', 'Rd_Width', 'Rd_Cond', 'Envinmt','Pos_Acc']

places[float_list] = places[float_list].astype(int, errors = 'raise')

print(places.isna().sum())
print()
print(places.info())
print()
print(places.shape)

Num_Acc             0
Rd_Cat              0
Traf_Direct         0
Lanes               0
Landmark            0
Dist_to_Landmark    0
Add_Lanes           0
Rd_Prof             0
Rd_Plan             0
Gre_Verge           0
Rd_Width            0
Rd_Cond             0
Envinmt             0
Pos_Acc             0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1121571 entries, 0 to 1121570
Data columns (total 14 columns):
 #   Column            Non-Null Count    Dtype
---  ------            --------------    -----
 0   Num_Acc           1121571 non-null  int64
 1   Rd_Cat            1121571 non-null  int32
 2   Traf_Direct       1121571 non-null  int32
 3   Lanes             1121571 non-null  int32
 4   Landmark          1121571 non-null  int32
 5   Dist_to_Landmark  1121571 non-null  int32
 6   Add_Lanes         1121571 non-null  int32
 7   Rd_Prof           1121571 non-null  int32
 8   Rd_Plan           1121571 non-null  int32
 9   Gre_Verge         1121571 non-null  int32
 

## Characteristics Dataset

### Translating the variable names from French to English

In [32]:
# Translation of the variable nacmes from French to English, also improving the names so that it becomes clearer, what they are about
characteristics.rename(columns={'an': 'year', 'mois':'month', 'jour': 'day', 'hrmn':'hhmm', 
                                'lum': 'daylight', 'agg': 'built-up_area', 'int':'intersection_category', 'atm': 'atmospheric_conditions',
                                'col': 'collision_category', 'com': 'municipality', 'adr':'adress', 'gps': 'gps_origin', 'lat': 'latitude',
                                'long': 'longitude', 'dep': 'department'}, inplace=True)

# Change the values for 'built-up_area' to make it more understandable, 1 means the accident happened in a built-up area and 0 means happened elsewhere. 
characteristics['built-up_area'].replace({1:0, 2:1}, inplace=True)

### Fixing incoherent format of year variable

In [33]:
characteristics['year'].value_counts()

5       87026
7       83850
6       82993
8       76767
9       74409
10      69379
11      66974
12      62250
17      60701
14      59854
16      59432
2019    58840
15      58654
13      58397
18      57783
2021    56518
2020    47744
Name: year, dtype: int64

The year format is inconsistent. Until 2018, the year was relative to the year 2000, e.g. "5" for 2005. This changed, however, in 2019 which was labeled as 2019.
We will change the year format to YYYY.

In [34]:
characteristics['year'].replace({5:2005, 6:2006, 7:2007, 8:2008, 9:2009, 10:2010, 11:2011,
                                                         12:2012, 13:2013, 14:2014, 15:2015, 16:2016, 17:2017, 18:2018}, inplace=True)

### Fix inconsistent time format

The time format inconsistent, sometimes it is hhmm, and sometimes hh:mm. We will therefore remove any ":" from the column 

In [35]:
#remove ':' from hhmm
characteristics['hhmm'] = characteristics['hhmm'].apply(lambda s: int(str(s).replace(':','')))

### Get weekday and weekend feature

In [36]:
characteristics['date'] = pd.to_datetime({'year':characteristics['year'],
                                                                 'month':dict_of_category_dfs['characteristics']['month'],
                                                                 'day':dict_of_category_dfs['characteristics']['day']})

In [37]:
# New variable: weekday, integer from 0 to 6 representing the weekdays from monday to sunday.
characteristics['day_of_week'] = dict_of_category_dfs['characteristics']['date'].apply(lambda x: x.day_of_week)

# New binary variable: is_weekend, 0 for monday to friday and 1 for saturday and sunday
characteristics['is_weekend'] = (dict_of_category_dfs['characteristics']['day_of_week'] > 4).astype('int')

### Remove trailing zeroes from Department variable
The Department codes are followed by a zero for the years 2005--2018, which has no practical use for us. We will therefore eliminate these trailing zeroes.
Also, since 2019 all the data is saved as strings. We will convert everything to strings, as this is nominal data, we will not make any calculations with it.

In [38]:
def department_converter(dep):
    # Takes in a department code as int and returns a string
    # e.g. 750 will be '75' for Paris
    # and 201 will be '2B'
    if dep == 201:
        return '2A'
    elif dep == 202:
        return '2B'
    elif dep>970:
        return str(dep)
    else:
        return str(dep).rstrip('0')

characteristics.loc[(np.less(characteristics['year'],2019)),'department'] = \
    characteristics[(np.less(characteristics['year'],2019))]['department'].apply(department_converter)

### Remove leading zeros from department code
The dataset from 2021 contains leading zeroes for the department codes 1 to 9. These have to be replaced.

In [39]:
characteristics['department'] = characteristics['department'].apply(lambda code: code.lstrip('0'))

### Fill missing values in atmospheric conditions variable

In [40]:
characteristics['atmospheric_conditions'] = characteristics['atmospheric_conditions'].fillna(
    characteristics['atmospheric_conditions'].mode()[0])

### Fill missing values in collision category variable

In [41]:
characteristics['collision_category'] = characteristics['collision_category'].fillna(
    characteristics['collision_category'].mode()[0])

## Vehicles dataset

### Translating the variable names from French to English

In [42]:
vehicles = vehicles.rename(columns = {'id_vehicule' : 'id_veh' , 'num_veh' : 'num_veh' ,
                           'senc' : 'direction' , 'catv' : 'cat_veh', 'obs' : 'obstacle', 'obsm' : 'obstacle_movable' ,
                          'choc' : 'initial_point' , 'manv' : 'principal_maneuver' , 'motor' : 'motor_veh', 'occutc' : 'num_occupants'})
vehicles.columns

Index(['Num_Acc', 'direction', 'cat_veh', 'num_occupants', 'obstacle',
       'obstacle_movable', 'initial_point', 'principal_maneuver', 'num_veh',
       'id_veh', 'motor_veh'],
      dtype='object')

### Check of the variables with the most missing values

Variable num_occupants is representing amount of passangers being victims of an accident when they used public transport system. Missing values are caused by not recording value 0 and keeping the cell empty. For this reason we decided to replace the missing values by 0.

In [43]:
vehicles["num_occupants"] = vehicles["num_occupants"].fillna(0)
vehicles['num_occupants'].isna().sum()

0

In [44]:
vehicles['num_occupants'].value_counts()

0.0      1901517
1.0         7079
2.0         1033
10.0         654
3.0          544
          ...   
97.0           1
73.0           1
102.0          1
82.0           1
78.0           1
Name: num_occupants, Length: 124, dtype: int64

The variable motor_veh represents the type of the motorisation of the vehicle. There are 85 % missing values in this column. Some of the values of this variable don't specificate an exact type but are tracked as unspecified, unknown, or other. We have decided to drop this variable as it doesn't have any significant influence on the target variable. 

In [45]:
vehicles = vehicles.drop(columns=['motor_veh'])

8 Variables have <= 1% missing information, so for those it should be fine to set the missing information just to zero.

In [46]:
vehicles[['Num_Acc', 'direction', 'cat_veh', 'obstacle', 'obstacle_movable', 'initial_point', 'principal_maneuver']] = vehicles[['Num_Acc', 'direction', 'cat_veh', 'obstacle', 'obstacle_movable', 'initial_point', 'principal_maneuver']].fillna(0)
vehicles.isna().sum()

Num_Acc                     0
direction                   0
cat_veh                     0
num_occupants               0
obstacle                    0
obstacle_movable            0
initial_point               0
principal_maneuver          0
num_veh                     0
id_veh                1635811
dtype: int64

# Merge all datasets

## Ensure Correct Attribution of Users to Vehicles

In [47]:
users['id_vehicule'].fillna(users['num_veh'], inplace=True)
users.drop(columns=['num_veh'], inplace=True)
users.rename(columns={'id_vehicule': 'id_veh'}, inplace=True)
users.set_index(['Num_Acc', 'id_veh'], inplace=True)

## Left Join for further investigations
We will continue working with the left join of the data, as the missing lines miss the most important variables anyway.

In [48]:
df = users.merge(vehicles, how='left', left_index=True, right_on=['Num_Acc', 'id_veh']) \
     .merge(characteristics, how='left', on='Num_Acc') \
     .merge(places, how='left', on='Num_Acc')

print(na_percentage(df))

place                      0.000000
User_category              0.000000
Severity                   0.000000
Sex                        0.000000
Trajectory                 0.000000
secu                      14.640663
LOCP                       0.000000
ACTP                       0.000000
StateP                     0.000000
YoB                        0.000000
secu1                     85.359337
secu2                     85.359337
SecuA                     14.640663
SecuB                     14.640663
Security                   0.000000
Num_Acc                    0.000000
direction                 85.359337
cat_veh                   85.359337
num_occupants             85.359337
obstacle                  85.359337
obstacle_movable          85.359337
initial_point             85.359337
principal_maneuver        85.359337
num_veh                   85.359337
id_veh                     0.000000
year                       0.000000
month                      0.000000
day                        0

In [49]:
df['Age'] = df['year'] - df['YoB']

In [50]:
del characteristics, places, vehicles, users, dict_of_category_dfs

## One-Hot Encoding of Categorical Features

In [51]:
df = pd.get_dummies(df.sample(frac=0.1, random_state=23),
       columns=['daylight', 'built-up_area', 'intersection_category', 
              'atmospheric_conditions', 'collision_category', 'department',
              'Rd_Cat', 'Traf_Direct', 'Add_Lanes', 'Rd_Prof', 'Rd_Plan', 
              'Rd_Cond', 'Envinmt', 'Pos_Acc', 'place', 
              'User_category', 'Sex', 'Trajectory', 'LOCP', 'ACTP', 
              'StateP', 'direction', 
              'cat_veh', 'obstacle', 'obstacle_movable', 'initial_point', 'principal_maneuver'])

# Export DataFrame to Pickle 
This step is necessary to be able to work with the data in another notebook.

In [52]:
df.to_pickle('../data/df.p')

The pickle file is too big to track on github, we will therefore create a second file which contains the output of the describe-method as well as the number of nans for each column and the dtypes of the DataFrame.

In [53]:
df_check_info = pyfra.df_testing_info(df)
df_check_info.to_csv('../data/df_check_info.csv')